> conv提特征 向量点积往大收敛 提取的特证逐渐与原图相似

In [139]:
# demo
import torch
import torch.nn as nn
import torch.nn.functional as F
in_channels = 1
out_channels = 1
kernel_size = 3 #传入元组 卷积核可以为矩形
batch_size = 2
bias = False
input_size = [batch_size, in_channels, 4, 4]
# 实例化类 首字母大写
conv_layer_test = torch.nn.Conv2d(3, 9, kernel_size, bias=bias)
# padding = sample
conv_layer = torch.nn.Conv2d(1, 1, kernel_size, bias=bias)
print(conv_layer.weight)
x = torch.randn(input_size)
x = F.dropout(x)
x = conv_layer(x)
print(conv_layer_test.weight.shape)
print(x)

Parameter containing:
tensor([[[[-0.2349, -0.2782, -0.3082],
          [ 0.2837,  0.2558,  0.1220],
          [ 0.0704,  0.2870, -0.1825]]]], requires_grad=True)
torch.Size([9, 3, 3, 3])
tensor([[[[-0.8537, -0.2350],
          [ 0.6817, -1.1483]]],


        [[[ 0.8677, -0.4955],
          [-0.0107,  0.7152]]]], grad_fn=<ConvolutionBackward0>)


> 函数实现

In [140]:
# 用原始的矩阵运算实现二维卷积,先不考虑batchsize维度和channel维数
def matrix_multiplication_for_conv2d(input, kernel, bias=None, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding))
    input_h, input_w = input.shape
    kernel_h, kernel_w = kernel.shape
    print(input.size(0))
    output_w = int(((input_w - kernel_w)/stride + 1))
    output_h = int(((input_h - kernel_h)/stride + 1))
    print(output_w)
    output = torch.zeros(output_w, output_h)
    for i in range(0, output_h, stride):
        for j in range(0, output_w, stride):
            # 方法1
            # p1d = (i, input_w -kernel_w-i,stride*i, input_h -kernel_h-stride*i)
            # kernel_pad = F.pad(kernel, p1d, 'constant', 0)
            # # print(kernel_pad)
            # output[int(i/stride), int(j/stride)] = torch.sum(torch.mul(kernel_pad, input))、
            # 方法2
            region = input[i:i+kernel_h,j:j+kernel_w]
            output[int(i/stride), int(j/stride)] = torch.sum(region * kernel) + bias
    return output
# bias = torch.randn(1)
x = torch.randn(6, 6)
kernel = torch.randn(3, 3)
output = matrix_multiplication_for_conv2d(x, kernel)
print(output)
pytorch_api_output = F.conv2d(x.reshape(1,1,x.size(0),x.size(1)),kernel.reshape(1,1,kernel.size(0),kernel.size(1)))
print(pytorch_api_output)

6
4


TypeError: unsupported operand type(s) for +: 'Tensor' and 'NoneType'